## Convert files from Thermo

ThermoRawFileParser https://github.com/compomics/ThermoRawFileParser

Convert profile MS1 data from Thermo raw files to centroid (MS1 and MS2) mzml

In [ ]:
!mono resources/ThermoRawFileParser/ThermoRawFileParser.exe -d="data/raw/" -o="results/"

In [ ]:
!(cd data/raw && ls *.raw > filelist.txt)
import pandas as pd
header_list = ["sample_name"]
df=pd.read_csv("data/raw/filelist.txt", names=header_list)
df["sample_name"]=df["sample_name"].replace(".raw", value="", regex=True)
df["comment"] = ""
df["MAPnumber"] = ""
df.to_csv("data/raw/samples.tsv", sep="\t")
df

## Import libraries

In [51]:
from pyopenms import *
from pandas import DataFrame
import pandas as pd
import pyteomics
from pyteomics.openms import featurexml
import numpy as np
import os
import glob
from pyteomics import mztab

## Preprocessing step

The first preprocessing function consists of six steps:

### 1) PrecursorCorrection (To the "highest intensity MS1 peak")

The highestintensity is used directly right after the file introduction, in order to correct any wrong MS1 precursor annotation - this means it will correct in the precursor window (e.g. 0.01 Da) and then use the highest intensity peak as the precursor. This is to correct for any instrument error. Since we assume that in the given mass window, the precursor with the hightest intensity was actually fragmented instead of any other (top-n method). This method is by default used in the Thermo Orbitrap instrument we have.

openMS definition:

"Selection of the peak with the highest intensity as corrected precursor mass in a given mass range (e.g. precursor mass +/- 0.2 Da)

For each MS2 spectrum the corresponding MS1 spectrum is determined by using the rt information of the precursor. In the MS1, the peak with the highest intensity in a given mass range to the uncorrected precursor m/z is selected and used as corrected precursor m/z."



In [54]:
path= "results/interim"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

input_files = glob.glob('results/*.mzML')
    
for filename in input_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    delta_mzs= []
    mzs = []
    rts= []
    PrecursorCorrection.correctToHighestIntensityMS1Peak(exp, 100.0, True, delta_mzs, mzs, rts)
    mzmlfile_path = os.path.join(os.path.dirname(filename), "interim", "", 'precursorcorrected_' + os.path.basename(filename))
    MzMLFile().store(mzmlfile_path, exp)

In [55]:
path= "results/features/interim"
isExist= os.path.exists(path)
if not isExist:
    os.mkdir(path)

input_files = glob.glob('results/interim/*.mzML')

# 2) Mass trace detection

#A mass trace extraction method that gathers peaks similar in m/z and moving along retention time.
#Peaks of a MSExperiment are sorted by their intensity and stored in a list of potential chromatographic apex positions. Only peaks that are above the noise threshold (user-defined) are analyzed and only peaks that are n times above this minimal threshold are considered as apices. This saves computational resources and decreases the noise in the resulting output.
#Starting with these, mass traces are extended in- and decreasingly in retention time. During this extension phase, the centroid m/z is computed on-line as an intensity-weighted mean of peaks.
#The extension phase ends when either the frequency of gathered peaks drops below a threshold (min_sample_rate, see MassTraceDetection parameters) or when the number of missed scans exceeds a threshold (trace_termination_outliers, see MassTraceDetection parameters).
#Finally, only mass traces that pass a filter (a certain minimal and maximal length as well as having the minimal sample rate criterion fulfilled) get added to the result.

for filename in input_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)

# 3) Elution peak detection

#Extracts chromatographic peaks from a mass trace.
#Mass traces may consist of several consecutively (partly overlapping) eluting peaks, e.g., stemming from (almost) isobaric compounds that are separated by retention time. Especially in metabolomics, isomeric compounds with exactly the same mass but different retentional behaviour may still be contained in the same mass trace.
#This method first applies smoothing on the mass trace's intensities, then detects local minima/maxima in order to separate the chromatographic peaks from each other. Detection of maxima is performed on the smoothed intensities and uses a fixed peak width (given as parameter chrom_fwhm) within which only a single maximum is expected. Currently smoothing is done using SavitzkyGolay smoothing with a second order polynomial and a frame length of the fixed peak width.
#Depending on the "width_filtering" parameters, mass traces are filtered by length in seconds ("fixed" filter) or by quantile.
#The output of the algorithm is a set of chromatographic peaks for each mass trace, i.e. a vector of split mass traces (see ElutionPeakDetection parameters).
#In general, a user would want to call the "detectPeaks" functions, potentially followed by the "filterByPeakWidth" function.
#This method in other words is "deconvolution".

    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
     
    if (epd.getParameters().getValue("width_filtering") == "auto"):
          epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
          mass_traces_final = mass_traces_split

# 4) Feature detection

#FeatureFinderMetabo assembles metabolite features from singleton mass traces.
#Mass traces alone would allow for further analysis such as metabolite ID or statistical evaluation. However, in general, monoisotopic mass traces are accompanied by satellite C13 peaks and thus may render the analysis more difficult. FeatureFinderMetabo fulfills a further data reduction step by assembling compatible mass traces to metabolite features (that is, all mass traces originating from one metabolite). To this end, multiple metabolite hypotheses are formulated and scored according to how well differences in RT (optional), m/z or intensity ratios match to those of theoretical isotope patterns.
#If the raw data scans contain the scan polarity information, it is stored as meta value "scan_polarity" in the output file.
#Mass trace clustering can be done using either 13C distances or a linear model (Kenar et al) – see parameter 'ffm:mz_scoring_13C'. Generally, for lipidomics, use 13C, since lipids contain a lot of 13C. For general metabolites, the linear model is usually more appropriate. To decide what is better, the total number of features can be used as indirect measure.
#the lower(!) the better (since more mass traces are assembled into single features). Detailed information is stored in the featureXML output: it contains meta-values for each feature about the mass trace differences (inspectable via TOPPView). 
#By default, the linear model is used.
    
    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "false")
    ffm_par.setValue("mz_scoring_by_elements", "false")
    ffm_par.setValue("report_convex_hulls", "true")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    feature_map_FFM.setUniqueIds()
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()])  


# 5) Metabolite adduct decharger (MetaboliteFeatureDeconvolution)

#For each peak, this algorithm reconstructs neutral masses by enumerating all possible adducts with matching charge. 
#You can add the list of adduct files and database files for the algorithm to parse through.
#With SIRIUS, an algorithm that is later used, you are only able to use singly charged adducts so charges higher than 1 are filtered out.

    mfd = MetaboliteFeatureDeconvolution()
    mdf_par = mfd.getDefaults()
    mdf_par.setValue("potential_adducts",  [b"H:+:0.6",b"Na:+:0.2",b"NH4:+:0.1", b"H2O:-:0.1"])
    mdf_par.setValue("charge_min", 1, "Minimal possible charge")
    mdf_par.setValue("charge_max", 1, "Maximal possible charge")
    mdf_par.setValue("charge_span_max", 1)
    mdf_par.setValue("max_neutrals", 1)
    mfd.setParameters(mdf_par)
        
    feature_map_DEC = FeatureMap()
    cons_map0 = ConsensusMap()
    cons_map1 = ConsensusMap()
    mfd.compute(feature_map_FFM, feature_map_DEC, cons_map0, cons_map1)
    featurefile = os.path.join("results", "", "features", "", "interim", "", 'MFD_' + os.path.basename(filename)[:-5] +".featureXML")
    FeatureXMLFile().store(featurefile, feature_map_DEC)


    Progress of 'mass trace detection':

    -- done [took 0.50 s (CPU), 0.07 s (Wall)] -- 

    Progress of 'elution peak detection':

    -- done [took 0.53 s (CPU), 0.07 s (Wall)] -- 

    Progress of 'assembling mass traces to features':

    -- done [took 0.40 s (CPU), 0.05 s (Wall)] -- 
MassExplainer table size: 12
5079 spectra and 1 chromatograms stored.
5089 spectra and 1 chromatograms stored.
5212 spectra and 1 chromatograms stored.
Generating Masses with threshold: -6.90776 ...
done
0 of 195 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 195 to 195
Found 195 putative edges (of 7012) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.026024 seconds,  with objective value: 10.26.
ILP score is: 10.26
Agreeing charges: 65/344


Compomer: Da -41.0003; q_net 0; logP -3.91202[[ (H2O1Na1) --> () ]]
 Compomer: Da -36.0449; q_net 0; logP -4.60517[[ (H2O1H4N1) --> () ]]
 Compomer: Da -21.9819; q_net 0; logP -2.12026[[ (Na1) --> (H1) ]]
 Compomer: Da -19.0184; q_net 0; logP -2.81341[[ (H1H2O1) --> () ]]
 Compomer: Da -17.0265; q_net 0; logP -2.81341[[ (H4N1) --> (H1) ]]
 Compomer: Da -4.9554; q_net 0; logP -3.91202[[ (Na1) --> (H4N1) ]]
 Compomer: Da 4.9554; q_net 0; logP -3.91202[[ (H4N1) --> (Na1) ]]
 Compomer: Da 17.0265; q_net 0; logP -2.81341[[ (H1) --> (H4N1) ]]
 Compomer: Da 19.0184; q_net 0; logP -2.81341[[ () --> (H1H2O1) ]]
 Compomer: Da 21.9819; q_net 0; logP -2.12026[[ (H1) --> (Na1) ]]
 Compomer: Da 36.0449; q_net 0; logP -4.60517[[ () --> (H2O1H4N1) ]]
 Compomer: Da 41.0003; q_net 0; logP -3.91202[[ () --> (H2O1Na1) ]]
 


    Progress of 'mass trace detection':

    -- done [took 0.48 s (CPU), 0.08 s (Wall)] -- 

    Progress of 'elution peak detection':

    -- done [took 0.62 s (CPU), 0.13 s (Wall)] -- 

    Progress of 'assembling mass traces to features':

    -- done [took 0.52 s (CPU), 0.10 s (Wall)] -- 
MassExplainer table size: 12
Generating Masses with threshold: -6.90776 ...
done
0 of 190 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 190 to 190
Found 190 putative edges (of 6805) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.024536 seconds,  with objective value: 9.11.
ILP score is: 9.11
Agreeing charges: 52/340

    Progress of 'mass trace detection':
      18.47 %               
    -- done [took 0.51 s (CPU), 0.08 s (Wall)] -- 

    Progress of 'elution peak detection':

    -- done [took 0.76 s (CPU), 0.11 s (Wall)] -- 


Compomer: Da -41.0003; q_net 0; logP -3.91202[[ (H2O1Na1) --> () ]]
 Compomer: Da -36.0449; q_net 0; logP -4.60517[[ (H2O1H4N1) --> () ]]
 Compomer: Da -21.9819; q_net 0; logP -2.12026[[ (Na1) --> (H1) ]]
 Compomer: Da -19.0184; q_net 0; logP -2.81341[[ (H1H2O1) --> () ]]
 Compomer: Da -17.0265; q_net 0; logP -2.81341[[ (H4N1) --> (H1) ]]
 Compomer: Da -4.9554; q_net 0; logP -3.91202[[ (Na1) --> (H4N1) ]]
 Compomer: Da 4.9554; q_net 0; logP -3.91202[[ (H4N1) --> (Na1) ]]
 Compomer: Da 17.0265; q_net 0; logP -2.81341[[ (H1) --> (H4N1) ]]
 Compomer: Da 19.0184; q_net 0; logP -2.81341[[ () --> (H1H2O1) ]]
 Compomer: Da 21.9819; q_net 0; logP -2.12026[[ (H1) --> (Na1) ]]
 Compomer: Da 36.0449; q_net 0; logP -4.60517[[ () --> (H2O1H4N1) ]]
 Compomer: Da 41.0003; q_net 0; logP -3.91202[[ () --> (H2O1Na1) ]]
 


    Progress of 'assembling mass traces to features':

    -- done [took 0.64 s (CPU), 0.09 s (Wall)] -- 


Compomer: Da -41.0003; q_net 0; logP -3.91202[[ (H2O1Na1) --> () ]]
 Compomer: Da -36.0449; q_net 0; logP -4.60517[[ (H2O1H4N1) --> () ]]
 Compomer: Da -21.9819; q_net 0; logP -2.12026[[ (Na1) --> (H1) ]]
 Compomer: Da -19.0184; q_net 0; logP -2.81341[[ (H1H2O1) --> () ]]
 Compomer: Da -17.0265; q_net 0; logP -2.81341[[ (H4N1) --> (H1) ]]
 Compomer: Da -4.9554; q_net 0; logP -3.91202[[ (Na1) --> (H4N1) ]]
 Compomer: Da 4.9554; q_net 0; logP -3.91202[[ (H4N1) --> (Na1) ]]
 Compomer: Da 17.0265; q_net 0; logP -2.81341[[ (H1) --> (H4N1) ]]
 Compomer: Da 19.0184; q_net 0; logP -2.81341[[ () --> (H1H2O1) ]]
 Compomer: Da 21.9819; q_net 0; logP -2.12026[[ (H1) --> (Na1) ]]
 Compomer: Da 36.0449; q_net 0; logP -4.60517[[ () --> (H2O1H4N1) ]]
 Compomer: Da 41.0003; q_net 0; logP -3.91202[[ () --> (H2O1Na1) ]]
 

MassExplainer table size: 12
Generating Masses with threshold: -6.90776 ...
done
0 of 262 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 262 to 262
Found 262 putative edges (of 9319) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.032995 seconds,  with objective value: 13.26.
ILP score is: 13.26
Agreeing charges: 96/472


In [56]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

class FeatureMapDF(FeatureMap):
    def __init__(self):
        super().__init__()

    def get_df(self):
        def gen(fmap: FeatureMap, fun):
            for f in fmap:
                yield from fun(f)

        def extractMetaData(f: Feature):
            # subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            bb = f.getConvexHull().getBoundingBox2D()
            if len(pep) != 0:
                hits = pep[0].getHits()
                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    # TODO what else
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], bb[0][1], bb[1][1], f.getOverallQuality(), f.getIntensity()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], bb[0][1], bb[1][1], f.getOverallQuality(), f.getIntensity()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], bb[0][1], bb[1][1], f.getOverallQuality(), f.getIntensity()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('RTstart', 'f'), ('RTend', 'f'), ('mzstart', 'f'), ('mzend', 'f'),
                    ('quality', 'f'), ('intensity', 'f')]
        mdarr = np.fromiter(iter=gen(self, extractMetaData), dtype=mddtypes, count=cnt)
        df= pd.DataFrame(mdarr).set_index('id').sort_values("mz").drop(columns= "sequence")
        #df= df[df["num_of_masstraces"]>=2.0]
        return df

In [66]:
input_files = glob.glob('results/features/interim/*.featureXML')

for filename in input_files:
    fmap = FeatureMapDF()
    FeatureXMLFile().load(filename, fmap)
    DF= fmap.get_df()
    feature_csv= os.path.join("results", "", "features", "", 'features_' + os.path.basename(filename)[23:-10] +"csv")
    DF.to_csv(feature_csv)
    display(DF)

,sequence,charge,RT,mz,RTstart,RTend,mzstart,mzend,quality,intensity
id,,,,,,,,,,
14574381195646092574,None,0,44.359695,151.625565,43.384243,57.545616,151.625458,151.625595,0.000203,2.224836e+06
16338075967608041563,None,0,44.359695,157.108429,43.384243,51.544956,157.108231,157.108521,0.000009,1.029335e+05
8928835633060465229,None,0,44.359695,158.961655,43.384243,52.779896,158.961227,158.964111,0.000072,7.918421e+05
643994329477609825,None,0,44.359695,173.092102,43.384243,51.544956,173.092041,173.092316,0.000016,1.803432e+05
14354833002193260213,None,0,44.359695,175.118988,43.384243,61.150822,175.117355,175.119049,0.000749,8.188940e+06
...,...,...,...,...,...,...,...,...,...,...
12756427979284513550,None,0,656.731079,203.219833,523.629150,659.959290,203.219772,203.219955,0.000100,1.094040e+06
6839185102285696090,None,0,656.731079,218.211411,564.371277,659.959290,218.211319,218.211517,0.000085,9.287049e+05
15769999653068254338,None,0,656.731079,232.061325,551.610107,659.959290,232.061172,232.061401,0.000086,9.443300e+05


,sequence,charge,RT,mz,RTstart,RTend,mzstart,mzend,quality,intensity
id,,,,,,,,,,
17460674035617910915,None,0,44.330742,213.123123,43.802853,57.673134,213.122498,213.123383,0.000079,1.032558e+06
2249926675916133177,None,0,44.330742,243.145111,43.802853,52.873592,243.144577,243.145126,0.000158,2.054517e+06
15185926433107649498,None,0,44.330742,829.300781,43.802853,46.818825,829.300354,829.301453,0.000018,2.407556e+05
5076819905368156000,None,1,45.570667,159.076416,43.802853,57.673134,159.076294,159.076447,0.001279,1.667603e+07
15224402655999378891,None,1,45.570667,176.102936,42.922054,55.309498,176.102890,176.105606,0.000867,1.130243e+07
...,...,...,...,...,...,...,...,...,...,...
15478353518190672299,None,1,652.327271,380.336914,649.475342,654.680115,380.336853,380.336945,0.000020,2.656234e+05
15415352146593844021,None,0,656.224060,232.061325,653.084351,659.937622,232.061249,232.061386,0.000015,1.948958e+05
16974107618006231698,None,1,657.477539,202.216522,468.327393,659.937622,202.216476,203.219955,0.000847,1.046033e+07


,sequence,charge,RT,mz,RTstart,RTend,mzstart,mzend,quality,intensity
id,,,,,,,,,,
6361836906925626596,None,0,44.448780,180.086639,43.366207,56.550369,180.086578,180.086685,0.000006,9.499561e+04
968751725736833827,None,0,44.448780,198.123703,43.366207,55.347057,198.123688,198.123886,0.000047,7.087062e+05
17051214398872344662,None,0,44.448780,232.140335,43.366207,54.108829,232.140198,232.140442,0.000011,1.607005e+05
13416262893556196560,None,0,44.448780,259.128784,43.366207,54.108829,259.128632,259.128906,0.000033,4.948526e+05
15890478133816133656,None,0,44.448780,276.155212,43.366207,61.412209,276.154144,276.155365,0.000028,4.270514e+05
...,...,...,...,...,...,...,...,...,...,...
5601161741334494701,None,0,656.273071,232.061325,471.926727,659.784668,232.061218,232.061539,0.000104,1.576343e+06
13644503181796914930,None,0,656.730164,210.079422,546.635376,659.784668,210.079391,210.079453,0.000483,7.336604e+06
14756344528618824088,None,0,658.370483,218.211411,547.842834,659.784668,218.211319,218.211487,0.000064,9.724687e+05


### 6) SIRIUS Adapter + CSI:Finger ID

#### (i)PrecursorCorrection (To the "nearest feature”)

    This algorithm is used after feature detection, adduct grouping and even identification via accurate mass search. It basically allows the precursor correction on MS2 level. 

    Which means that if there are MS2 spectra in my feature space which have been measured in isotope traces, it “corrects” the MS2 spectrum annotation to the monoisotopic trace. That is why you have a high mass deviation 100 pm, but 0.0 rt tolerance. So it basically corrects the MS2 to the feature centroid that can be found/mapped by SIRIUS::preprocessing later on.

    Wrong assignment of the mono-isotopic mass for precursors are assumed:

    1. if precursor_mz matches the mz of a non-monoisotopic feature mass trace and 
    2. in the case that believe_charge is true: if feature_charge matches the precursor_charge In the case of wrong mono-isotopic assignment several options for correction are available: keep_original will create a copy of the precursor and tandem spectrum for the new mono-isotopic mass trace and retain the original one. all_matching_features does this not for only the closest feature but all features in a question.

#### We need to preprocess again without convex hulls this time, otherwise the results from SIRIUS are incorrect

#### (ii) SIRIUS Adapter
    The SIRIUS function is optional and includes the SIRIUS Adapter Algorithm from the Boecher lab. 


    The algorithm generates formula prediction from scores calculated from 1) MS2 fragmentation scores (ppm error + intensity) and 2) MS1 isotopic pattern scores.


    It can only compute feautures that are singly charged. There is also a timeout for compounds (compound timeout so that it doesn't compute for longer than 100 seconds per feature, which normally happens with larger molecules).


    -sirius:compound_timeout <number>                    
    Maximal computation time in seconds for a single compound. 0 for an infinite amount of time. (default: '100' min: '0')

        
    This algorith can help in data dereplication and analysis for direct library search. 

#### (iii) CSI:FingerID

The CSI_fingerID function is another algorithm from the Boecher lab, just like SIRIUS adapter and is using the formula predictions from SIRIUS, to search in structural libraries and predict the structure of each formula

If you replace: 
    
    out_csifingerid = os.path.join("results", "", "interim", "", "CSI", "", 'structures_' + os.path.basename(filename) +".mzTab")

with an empty string:

    out_csifingerid = ""
        
CSI:FingerID will be ignored and will not be computed.

In [70]:
path1= "results/SIRIUS/interim"
isExist= os.path.exists(path1)
if not isExist:
    os.mkdir(path1)

path2= "results/CSI/interim"
isExist= os.path.exists(path2)
if not isExist:
    os.mkdir(path2)

input_files = glob.glob('results/interim/*.mzML')

for filename in input_files:
    exp = MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    PrecursorCorrection.correctToNearestFeature(feature_map_DEC, exp, 0.0, 100.0, True, False, False, False, 3, 0)

    mass_traces = []
    mtd = MassTraceDetection()
    mtd_par = mtd.getDefaults()
    mtd_par.setValue("mass_error_ppm", 10.0) 
    mtd_par.setValue("noise_threshold_int", 1.0e04)
    mtd.setParameters(mtd_par)
    mtd.run(exp, mass_traces, 0)

    mass_traces_split = []
    mass_traces_final = []
    epd = ElutionPeakDetection()
    epd_par = epd.getDefaults()
    epd_par.setValue("width_filtering", "fixed")
    epd.setParameters(epd_par)
    epd.detectPeaks(mass_traces, mass_traces_split)
        
    if (epd.getParameters().getValue("width_filtering") == "auto"):
        epd.filterByPeakWidth(mass_traces_split, mass_traces_final)
    else:
        mass_traces_final = mass_traces_split

    feature_map_FFM = FeatureMap()
    feat_chrom = []
    ffm = FeatureFindingMetabo()
    ffm_par = ffm.getDefaults() 
    ffm_par.setValue("isotope_filtering_model", "none")
    ffm_par.setValue("remove_single_traces", "false")
    ffm_par.setValue("mz_scoring_by_elements", "false")
    ffm.setParameters(ffm_par)
    ffm.run(mass_traces_final, feature_map_FFM, feat_chrom)
    feature_map_FFM.setUniqueIds()
    feature_map_FFM.setPrimaryMSRunPath([filename.encode()])

    mfd = MetaboliteFeatureDeconvolution()
    mdf_par = mfd.getDefaults()
    mdf_par.setValue("potential_adducts",  [b"H:+:0.6",b"Na:+:0.2",b"NH4:+:0.1", b"H2O:-:0.1"])
    mdf_par.setValue("charge_min", 1, "Minimal possible charge")
    mdf_par.setValue("charge_max", 1, "Maximal possible charge")
    mdf_par.setValue("charge_span_max", 1)
    mdf_par.setValue("max_neutrals", 1)
    mfd.setParameters(mdf_par)
    feature_map_DEC = FeatureMap()
    cons_map0 = ConsensusMap()
    cons_map1 = ConsensusMap()
    mfd.compute(feature_map_FFM, feature_map_DEC, cons_map0, cons_map1)
    
    featureinfo = os.path.join("results", "", "SIRIUS", "", "interim", "", 'MFD_noconvexhulls_' + os.path.basename(filename)[:-5] + ".featureXML")
    FeatureXMLFile().store(featureinfo, feature_map_DEC)

    sirius_algo = SiriusAdapterAlgorithm()
    sirius_algo_par = sirius_algo.getDefaults()
    sirius_algo_par.setValue("preprocessing:filter_by_num_masstraces", 2) #Number of mass traces each feature has to have to be included
    sirius_algo_par.setValue("preprocessing:feature_only", "true") #Uses the feature information from in_featureinfo to reduce the search space to MS2
    sirius_algo_par.setValue("sirius:profile", "orbitrap")
    sirius_algo_par.setValue("sirius:db", "none")
    sirius_algo_par.setValue("sirius:ions_considered", "[M+H]+, [M-H2O+H]+, [M+Na]+, [M+NH4]+")
    sirius_algo_par.setValue("sirius:elements_enforced", "CHN[15]OS[4]") 
    sirius_algo_par.setValue("project:processors", 2)
    sirius_algo_par.setValue("fingerid:db", "BIO")
    sirius_algo.setParameters(sirius_algo_par)
        

    fm_info = FeatureMapping_FeatureMappingInfo()
    feature_mapping = FeatureMapping_FeatureToMs2Indices() 
    sirius_algo.preprocessingSirius(featureinfo,
                                    exp,
                                    fm_info,
                                    feature_mapping)
    sirius_algo.logFeatureSpectraNumber(featureinfo, 
                                        feature_mapping,
                                        exp)
    msfile = SiriusMSFile()
    debug_level = 3
    sirius_tmp = SiriusinterimoraryFileSystemObjects(debug_level)
    siriusstring= String(sirius_tmp.getTmpMsFile())
    feature_only = sirius_algo.isFeatureOnly()
    isotope_pattern_iterations = sirius_algo.getIsotopePatternIterations()
    no_mt_info = sirius_algo.isNoMasstraceInfoIsotopePattern()
    compound_info = []
    msfile.store(exp,
                String(sirius_tmp.getTmpMsFile()),
                feature_mapping, 
                feature_only,
                isotope_pattern_iterations, 
                no_mt_info, 
                compound_info)
    
    out_csifingerid = os.path.join("results", "", "CSI", "", "interim", "", 'structures_' + os.path.basename(filename)[19:-5] +".mzTab")
    executable= "resources/Sirius/sirius.app/Contents/MacOS/sirius"
    subdirs = sirius_algo.callSiriusQProcess(String(sirius_tmp.getTmpMsFile()),
                                            String(sirius_tmp.getTmpOutDir()),
                                            String(executable),
                                            String(out_csifingerid),
                                            False)
    candidates = sirius_algo.getNumberOfSiriusCandidates()
    sirius_result = MzTab()
    siriusfile = MzTabFile()
    SiriusMzTabWriter.read(subdirs,
                        filename,
                        candidates,
                        sirius_result)
    
    sirius_file= os.path.join("results", "", "SIRIUS", "", "interim", "",'formulas_' + os.path.basename(filename)[19:-5] +".mzTab")
    siriusfile.store(sirius_file, sirius_result)

HighResPrecursorMassCorrector warning: at least one feature has no convex hull - omitting feature for matching

    Progress of 'mass trace detection':

    -- done [took 0.06 s (CPU), 0.06 s (Wall)] -- 

    Progress of 'elution peak detection':

    -- done [took 0.53 s (CPU), 0.07 s (Wall)] -- 

    Progress of 'assembling mass traces to features':

    -- done [took 0.41 s (CPU), 0.05 s (Wall)] -- 
MassExplainer table size: 12
Generating Masses with threshold: -6.90776 ...
done
0 of 812 valid net charge compomer results did not pass the feature charge constraints
Inferring edges raised edge count from 812 to 812
Found 812 putative edges (of 33895) and avg hit-size of 1
Using solver 'coinor' ...
Optimal solution found!
 Branch and cut took 0.071544 seconds,  with objective value: 34.94.
ILP score is: 34.94
Agreeing charges: 199/1130
ConsensusFeature::computeDechargeConsensus() WARNING: Feature's charge is 0! This will lead to M=0!
<HighResPrecursorMassCorrector warning: at least one

Compomer: Da -41.0003; q_net 0; logP -3.91202[[ (H2O1Na1) --> () ]]
 Compomer: Da -36.0449; q_net 0; logP -4.60517[[ (H2O1H4N1) --> () ]]
 Compomer: Da -21.9819; q_net 0; logP -2.12026[[ (Na1) --> (H1) ]]
 Compomer: Da -19.0184; q_net 0; logP -2.81341[[ (H1H2O1) --> () ]]
 Compomer: Da -17.0265; q_net 0; logP -2.81341[[ (H4N1) --> (H1) ]]
 Compomer: Da -4.9554; q_net 0; logP -3.91202[[ (Na1) --> (H4N1) ]]
 Compomer: Da 4.9554; q_net 0; logP -3.91202[[ (H4N1) --> (Na1) ]]
 Compomer: Da 17.0265; q_net 0; logP -2.81341[[ (H1) --> (H4N1) ]]
 Compomer: Da 19.0184; q_net 0; logP -2.81341[[ () --> (H1H2O1) ]]
 Compomer: Da 21.9819; q_net 0; logP -2.12026[[ (H1) --> (Na1) ]]
 Compomer: Da 36.0449; q_net 0; logP -4.60517[[ () --> (H2O1H4N1) ]]
 Compomer: Da 41.0003; q_net 0; logP -3.91202[[ () --> (H2O1Na1) ]]
 

Number of features to be processed: 470
<ConsensusFeature::computeDechargeConsensus() WARNING: Feature's charge is 0! This will lead to M=0!> occurred 2684 times
No MS1 spectrum for this precursor. Occurred 0 times.
5 spectra were skipped due to precursor charge below -1 and above +1.
Mono charge assumed and set to charge 1 with respect to current polarity 93 times.
76 features were skipped due to feature charge below -1 and above +1.
Running SIRIUS with the following command line parameters: 
--processors=2 --input /private/var/folders/c_/ysz9v_bd1yb7h3ymmkn6m199jbv7x7/T/20211022_140958_vpn9081.extern.uni-tuebingen.de_53181_6.ms --project /private/var/folders/c_/ysz9v_bd1yb7h3ymmkn6m199jbv7x7/T/20211022_140958_vpn9081.extern.uni-tuebingen.de_53181_5/sirius_out sirius --db=none --elements-enforced=CHN[15]OS[4] --tree-timeout=0 --candidates=5 --profile=orbitrap --ppm-max=10.0 --compound-timeout=100 --ppm-max-ms2=10.0 --ions-considered=[M+H]+, [M-H2O+H]+, [M+Na]+, [M+NH4]+ fingerid --db=

RuntimeError: the file 'results/SIRIUS/temp/formulas_20210827_UMETAB219_POS_DNPM_Plate-2_MDNA_WGS_14_rep3.mzTab' could not be created. 

In [ ]:
input_files = glob.glob('results/SIRIUS/interim/*.mzTab')
for filename in input_files:
    sirius=  pyteomics.mztab.MzTab(filename, encoding='UTF8', table_format='df')
    sirius.metadata
    df= sirius.small_molecule_table
    SIRIUS_DF= df.drop(columns= ["identifier", "smiles", "inchi_key", "description", "calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
    sirius_DF_file= os.path.join("results", "", "SIRIUS", "", os.path.basename(filename)[-6] +".csv")
    SIRIUS_DF.to_csv(sirius_DF_file)

In [50]:
# Example formula predictions dataframe

SIRIUS_DF= pd.read_csv("./results/SIRIUS/formulas_Epemicins.csv", index_col= "Unnamed: 0")
SIRIUS_DF=SIRIUS_DF[SIRIUS_DF["opt_global_explainedIntensity"] >= 0.6] #opt_global_explainedIntensity should be higher than 0.8 or 0.9 even for reliable results
SIRIUS_DF= SIRIUS_DF.sort_values(by= "exp_mass_to_charge")

FileNotFoundError: [Errno 2] No such file or directory: './results/SIRIUS/formulas_Epemicins.csv'

In [ ]:
input_files = glob.glob('results/CSI/interim/*.mzTab')
for filename in input_files:
    CSI=  pyteomics.mztab.MzTab(filename, encoding='UTF8', table_format='df')
    CSI.metadata
    df= CSI.small_molecule_table
    csifingerID= df.drop(columns= ["calc_mass_to_charge", "charge", "taxid", "species","database", "database_version", "spectra_ref", "search_engine", "modifications"])
    csifingerID_file= os.path.join("results", "", "CSIfingerID", "", 'structures_' + os.path.basename(filename)[-6] +".csv")
    csifingerID.to_csv(csifingerID)

In [ ]:
#example formula predictions table for Epemicins
csifingerID= pd.read_csv("./results/CSI/interim/structures_Epemicins.csv",index_col= "Unnamed: 0")
csifingerID

,identifier,chemical_formula,smiles,inchi_key,description,exp_mass_to_charge,retention_time,best_search_engine_score[1],opt_global_rank,opt_global_compoundId,opt_global_compoundScanNumber,opt_global_featureId,opt_global_native_id,opt_global_adduct,opt_global_dblinks,opt_global_dbflags
0,14852549|14852550|25053655|24971283|53652498|5...,C25H45NO5,CCCCCCCCCCCC(CC1C(C(=O)O1)CCCCCC)OC(=O)CNC=O,ANPULBVRORHAGO,"Orlistat derivative, 15g",457.363953,327.701974,-167.839042,1,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,MeSH:(14852550)|PubChem:(14852549 14852550 250...,70
1,9889418|11059239,C25H45NO5,CCCCCCCCCCC1C(OC1=O)CC(CCCCCCC)OC(=O)CNC=O,RQFZMIXBLJEUGS,NaN,457.363953,327.701974,-168.290573,2,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,COCONUT:(CNP0260950)|KNApSAcK:(16423)|Natural ...,3178514
2,71464556|71464557,C25H45NO5,CCCCCC=CCC=CCCCCCC(CC(=O)OC(CC(=O)O)C[N+](C)(C...,WQYXCASYXUFNSI,3-hydroxylinoleylcarnitine,457.363953,327.701974,-170.979093,3,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,PubChem:(71464556 71464557)|CHEBI:(73075),34
3,9824494,C25H45NO5,CCCCCCCC(CC1C(C(=O)O1)CCCCCCCC(C)C)OC(=O)CNC=O,SMEWLQMERHEMJK,NaN,457.363953,327.701974,-171.211550,4,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,COCONUT:(CNP0157254)|Natural Products:(UNPD174...,3178498
4,3049003,C25H45NO5,CCCCCCC(C)(CCN1C(C(=O)C(C1=O)(C)C)CCCCCCC(=O)O...,OHQKFFPFFFUNJW,"Ethyl 4,4-dimethyl-3,5-dioxo-1-(3-hydroxy-3-me...",457.363953,327.701974,-250.749662,5,1571,1572,id_6254478322423518528,controllerType=0 controllerNumber=1 scan=1572,[M + H4 + N]+,PubChem:(3049003)|PubChem class - safety and t...,67108866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2572,288197|2754113|7056395|7056399|11868690|118686...,C19H28O5,CC12CCC3C(C1CCC2=O)CCC(C3(C)CC(=O)O)CC(=O)O,CEUQHWSIXGFNGI,NaN,337.199054,465.541832,-155.792453,1,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0068730)|PubChem:(288197 2754113 7...,3153922
2573,"""""",C19H28O5,CC(=O)C(=O)C1(CCC2C(C(=O)CCC2(C1CC(=O)O)C)(C)C)C,QWLGLRAGSFVBDI,NaN,337.199054,465.541832,-168.039526,2,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0136763)|Natural Products:(UNPD195...,3178496
2574,354385|494326|23425264,C19H28O5,CC(=O)OC1CC2CC3(C(CC(C3C2(C1=C)O)(C)C)OC(=O)C)C,CLIPYJQRTQNGAM,".delta.-(sup 9,8.beta., 10.alpha.-triol, 3,8-d...",337.199054,465.541832,-203.768295,3,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0216092)|Natural Products:(UNPD156...,3178498
2575,"""""",C19H28O5,CC(=O)OC1CC2CC3(CC(C(C3C2(C1=C)O)(C)C)OC(=O)C)C,KEVCQSXXJWIVSR,NaN,337.199054,465.541832,-205.873518,4,2336,2337,id_361053789030700068,controllerType=0 controllerNumber=1 scan=2337,[M + H]+,COCONUT:(CNP0261220)|Natural Products:(UNPD183...,2129920


#### Explanation of columns
##### mz= mass-to-charge ratio (m/z)
##### RT= retention time (sec)
##### intensity = intensity of the feature (AU-arbitrary units)
##### FWHM= Full Width of the peak at Half its Maximum height
##### num_of_masstraces	= number of mass traces detected (single mass traces are excluded). This is relevant to the isotopic pattern
##### isotope_distances = distance in mz between the isotopes (jumps of app. 1 is important to confirm that this is a real feature) 

### Create a metadata csv file for GNPS from the samples.tsv file 

In [ ]:
path= "results/GNPSexport/interim"
os.makedirs(path, exist_ok=False)

df= pd.read_csv("results/samples.tsv", sep= "\t", index_col= "Unnamed: 0")
metadata= df.rename(columns= {"sample_name": "filename", "comment": "ATTRIBUTE_comment", "MAPnumber": "ATTRIBUTE_MAPnumber"})
metadata["filename"]= metadata["filename"].astype(str) +".mzml"
metadata['ATTRIBUTE_MAPnumber'] = np.arange(len(metadata))
metadata["ATTRIBUTE_comment"]= "MAP" + metadata["ATTRIBUTE_MAPnumber"].astype(str)
metadata= metadata.drop(columns= "ATTRIBUTE_MAPnumber")
metadata['ATTRIBUTE_genomeID']=metadata['filename'].str.extract(r'(NBC_?\d*)')
metadata['ATTRIBUTE_genomeIDMDNA']=metadata['filename'].str.extract(r'(MDNA_WGS_?\d*)')
metadata['ATTRIBUTE_genomeID']=metadata['ATTRIBUTE_genomeID'].fillna(metadata['ATTRIBUTE_genomeIDMDNA'])
metadata=metadata.drop(columns="ATTRIBUTE_genomeIDMDNA")
#metadata['ATTRIBUTE_genomeID']= metadata['ATTRIBUTE_genomeID'].replace(to_replace= r'NBC', value= 'NBC_', regex= True)
#metadata['ATTRIBUTE_genomeID']= metadata['ATTRIBUTE_genomeID'].replace(to_replace= r'MDNAWGS', value= 'MDNA_WGS_', regex= True)
metadata.to_csv("results/GNPSexport/metadata.tsv", sep='\t')
metadata

From the CLI, I am using the following algorithms to generate files compatible to GNPS for FBMN: 

#### MapAlignerPoseClustering is used to perform a linear retention time alignment, basically correct for linear shifts in retention time. 
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_MapAlignerPoseClustering.html

In [20]:
# get in index of feature map with highest number of features in feature map list
input_files = glob.glob('results/features/interim/*.featureXML')
path= "results/consensus/interim/"
os.makedirs(path, exist_ok=False)

feature_maps = []
for featurexml_file in input_files:
    fmap = FeatureMap()
    FeatureXMLFile().load(featurexml_file, fmap)
    feature_maps.append(fmap)
    
ref_index = [i[0] for i in sorted(enumerate([fm.size() for fm in feature_maps]), key=lambda x:x[1])][-1]

aligner = MapAlignmentAlgorithmPoseClustering()
aligner_par= aligner.getDefaults()
aligner_par.setValue("max_num_peaks_considered", -1)
aligner.setParameters(aligner_par)
aligner.setReference(feature_maps[ref_index])

for feature_map in feature_maps[:ref_index] + feature_maps[ref_index+1:]:
    trafo = TransformationDescription()
    aligner.align(feature_map, trafo)
    transformer = MapAlignmentTransformer()
    transformer.transformRetentionTimes(feature_map, trafo, True) # store original RT as meta value
    feature_file = os.path.join("results", "", "consensus", "", "interim", "", 'MapAligned_' + os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[19:-5] +".featureXML")
    trafo_file= os.path.join("results", "", "consensus", "", "interim", "", 'MapAligned_' + os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[19:-5] +".trafoXML")
    FeatureXMLFile().store(feature_file, feature_map)
    TransformationXMLFile().store(trafo_file, trafo)

#### Introduce the features to a protein identification file (idXML)   
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_IDMapper.html

In [49]:
import pyopenms as pms
input_files= glob.glob("results/interim/*.mzML")
mapper = pms.IDMapper()
protein_ids = []
peptide_ids= []
id_file = "resources/emptyfile.idXML"
IdXMLFile().load(id_file, protein_ids, peptide_ids)
use_centroid_rt= True
use_centroid_mz= True

exp = MSExperiment()
for filename in input_files:
    MzMLFile().load(filename, exp)
for featurexml_file in feature_maps:
    idx = feat['id']
    for key in feat.keys():
        if key == 'id':
            mapper.annotate(featurexml_file, peptide_ids, protein_ids, use_centroid_rt, use_centroid_mz, exp)
            featureidx_file = os.path.join("results", "", "consensus", "", "interim", "", 'IDMapper_' + os.path.basename(feature_map.getMetaValue('spectra_data')[0].decode())[19:-5] +".featureXML")
            FeatureXMLFile().store(featureidx_file, featurexml_file)

Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 4523
Unidentified precursor assigned to exactly one feature: 0
Unidentified precursor assigned to multiple features: 0
Feature annotation with identifications:
    no ID: 3604
    single ID: 0
    multiple IDs (identical): 0
    multiple IDs (divergent): 0


Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 4523
Unidentified precursor assigned to exactly one feature: 0
Unidentified precursor assigned to multiple features: 0
Feature annotation with identifications:
    no ID: 3656
    single ID: 0
    multiple IDs (identical): 0
    multiple IDs (divergent): 0


Unassigned peptides: 0
Peptides assigned to exactly one feature: 0
Peptides assigned to multiple features: 0
Unassigned and unidentified precursors: 4523
Unidentified precursor assigne

#### The FeatureLinkerUnlabeledKD is used to aggregate the feature information (from single files) into a ConsensusFeature, linking features from different files together, which have a smiliar m/z and rt (no MS2 data).
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FeatureLinkerUnlabeledKD.html

In [35]:
class ConsensusMapDF(ConsensusMap):
    def __init__(self):
        super().__init__()

    def get_intensity_df(self):
        labelfree = self.getExperimentType() == "label-free"
        filemeta = self.getColumnHeaders()  # type: dict[int, ColumnHeader]
        labels = list(set([header.label for header in
                           filemeta.values()]))  # TODO could be more efficient. Do we require same channels in all files?
        files = list(set([header.filename for header in filemeta.values()]))
        label_to_idx = {k: v for v, k in enumerate(labels)}
        file_to_idx = {k: v for v, k in enumerate(files)}

        def gen(cmap: ConsensusMap, fun):
            for f in cmap:
                yield from fun(f)

        if not labelfree:
            # TODO write two functions for LF and labelled. One has only one channel, the other has only one file per CF
            def extractRowBlocksChannelWideFileLong(f: ConsensusFeature):
                subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
                filerows = defaultdict(lambda: [0] * len(labels))  # TODO use numpy array?
                for fh in subfeatures:
                    header = filemeta[fh.getMapIndex()]
                    row = filerows[header.filename]
                    row[label_to_idx[header.label]] = fh.getIntensity()
                return (f.getUniqueId(), filerows)

            def extractRowsChannelWideFileLong(f: ConsensusFeature):
                uniqueid, rowdict = extractRowBlocksChannelWideFileLong(f)
                for file, row in rowdict.items():
                    row.append(file)
                    yield tuple([uniqueid] + row)

            if len(labels) == 1:
                labels[0] = "intensity"
            dtypes = [('id', np.dtype('uint64'))] + list(zip(labels, ['f'] * len(labels)))
            dtypes.append(('file', 'U300'))
            # For TMT we know that every feature can only be from one file, since feature = PSM
            #cnt = 0
            #for f in self:
            #    cnt += f.size()

            intyarr = np.fromiter(iter=gen(self, extractRowsChannelWideFileLong), dtype=dtypes, count=self.size())
            return pd.DataFrame(intyarr).set_index('id')
        else:
            # Specialized for LabelFree which has to have only one channel
            def extractRowBlocksChannelLongFileWideLF(f: ConsensusFeature):
                subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
                row = [0.] * len(files)  # TODO use numpy array?
                for fh in subfeatures:
                    header = filemeta[fh.getMapIndex()]
                    row[file_to_idx[header.filename]] = fh.getIntensity()
                yield tuple([f.getUniqueId()] + row)

            dtypes = [('id', np.dtype('uint64'))] + list(zip(files, ['f'] * len(files)))
            # cnt = self.size()*len(files) # TODO for this to work, we would need to fill with NAs for CFs that do not go over all files
            cnt = self.size()

            intyarr = np.fromiter(iter=gen(self, extractRowBlocksChannelLongFileWideLF), dtype=dtypes, count=cnt)
            return pd.DataFrame(intyarr).set_index('id')

    def get_metadata_df(self):
        def gen(cmap: ConsensusMap, fun):
            for f in cmap:
                yield from fun(f)

        def extractMetaData(f: ConsensusFeature):
            # subfeatures = f.getFeatureList()  # type: list[FeatureHandle]
            pep = f.getPeptideIdentifications()  # type: list[PeptideIdentification]
            if len(pep) != 0:
                hits = pep[0].getHits()
                if len(hits) != 0:
                    besthit = hits[0]  # type: PeptideHit
                    # TODO what else
                    yield f.getUniqueId(), besthit.getSequence().toString(), f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()
                else:
                    yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()
            else:
                yield f.getUniqueId(), None, f.getCharge(), f.getRT(), f.getMZ(), f.getQuality()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('sequence', 'U200'), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('quality', 'f')]
        mdarr = np.fromiter(iter=gen(self, extractMetaData), dtype=mddtypes, count=cnt)
        return pd.DataFrame(mdarr).set_index('id')

In [39]:
feature_grouper = FeatureGroupingAlgorithmKD()

consensus_map = ConsensusMapDF()
file_descriptions = consensus_map.getColumnHeaders()

for i, feature_map in enumerate(feature_maps):
    file_description = file_descriptions.get(i, ColumnHeader())
    file_description.filename = feature_map.getMetaValue('spectra_data')[0].decode()
    file_description.size = feature_map.size()
    file_description.unique_id = feature_map.getUniqueId()
    file_descriptions[i] = file_description

consensus_map.setColumnHeaders(file_descriptions)
feature_grouper.group(feature_maps, consensus_map)


Consensus_file= os.path.join("results", "", "consensus", "","interim", "", 'consensus_' + os.path.basename(filename)[19:-5] +".consensusXML")
ConsensusXMLFile().store(Consensus_file, consensus_map)


# get intensities as a DataFrame
intensities = consensus_map.get_intensity_df()

# get meta data as DataFrame
meta_data = consensus_map.get_metadata_df()[['RT', 'mz', 'quality']]

# you can concatenate these two for a "result" DataFrame
result = pd.concat([meta_data, intensities], axis=1)

# if you don't need labeled index, remove it (and/or save with index = False)
result.reset_index(drop=True, inplace=True)

# store as tsv file
result.to_csv('results/consensus/Consensus.tsv', sep = '\t', index = False)


  Progress of 'computing RT transformations':

  -- done [took 0.13 s (CPU), 0.13 s (Wall)] -- 

  Progress of 'linking features':

  -- done [took 0.30 s (CPU), 0.29 s (Wall)] -- 
ConsensusXMLFile::store():  found 5365 invalid unique ids


RuntimeError: While storing '': ProteinIdentifications are not unique, which leads to loss of unique PeptideIdentification assignment. Duplicated Protein-ID is:UNKNOWN_SEARCH_RUN_IDENTIFIER.
The random chance of this error occurring is 1:2^64. Re-run the last tool and if the error occurs again, please report this as a bug in: 

#### Filter out the features that do not have an MS2 pattern
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_FileFilter.html

    rule FileFilter:
        input:
            "results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML"
        output:
            "results/Consensus/filtered.consensusXML"
        shell:
            """
            resources/OpenMS-2.7.0/bin/FileFilter -id:remove_unannotated_features -in {input} -out {output} 
            """


In [ ]:
!resources/OpenMS-2.7.0/bin/FileFilter -id:remove_unannotated_features -in results/Consensus/interim/FeatureLinkerUnlabeledKD.consensusXML -out results/Consensus/filtered.consensusXML


#### GNPS_export creates an mgf file with only the MS2 information of all files (introduce mzml files with spaces between them)
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/nightly/html/TOPP_GNPSExport.html

    rule GNPS_export:
        input:
            "results/Consensus/filtered.consensusXML",
            expand("results/{samples}/interim/precursorcorrected_{samples}.mzML", samples=SAMPLES)
        output:
            "results/GNPSexport/MSMS.mgf" 
        shell:
            """
            resources/OpenMS-2.7.0/bin/GNPSExport -in_cm {input[0]} -in_mzml {input[1]} -out {output} 
            """

In [ ]:
!resources/OpenMS-2.7.0/bin/GNPSExport -ini resources/GNPSExport.ini -in_cm results/Consensus/filtered.consensusXML -in_mzml results/Epemicins/interim/precursorcorrected_Epemicins.mzML results/GermicidinA/interim/precursorcorrected_GermicidinA.mzML results/GermicidinB/interim/precursorcorrected_GermicidinB.mzML -out results/GNPSexport/MSMS.mgf

Progress of 'parsing features and ms2 identifications...':
  1.63 %               GNPSExport took 2.62 s (wall), 2.58 s (CPU), 0.16 s (system), 2.42 s (user); Peak Memory Usage: 87 MB.


#### export the consensusXML file to a txt file for GNPS
Documentation: https://abibuilder.informatik.uni-tuebingen.de/archive/openms/Documentation/release/latest/html/TOPP_TextExporter.html
    
    rule txt_export:
        input:
            "results/Consensus/filtered.consensusXML"
        output:
            "results/GNPSexport/FeatureQuantificationTable.txt" 
        shell:
            """
            resources/OpenMS-2.7.0/bin/TextExporter -in {input} -out {output}
            """

In [ ]:
!resources/OpenMS-2.7.0/bin/TextExporter -in results/Consensus/filtered.consensusXML -out results/GNPSexport/FeatureQuantificationTable.txt

TextExporter took 0.09 s (wall), 0.09 s (CPU), 0.01 s (system), 0.08 s (user); Peak Memory Usage: 19 MB.


In [ ]:
!resources/OpenMS-2.7.0/bin/FeatureFinderMetaboIdent -in results/Epemicins/interim/precursorcorrected_Epemicins.mzML -id MetaboliteIdentification.tsv -out results/Epemicins/FFMI_Epemicins.featureXML

RT window size calculated as 240 seconds.
Loading targets and creating assay library...
Loading input LC-MS data...
Progress of 'loading spectra list':
-- done [took 0.45 s (CPU), 0.46 s (Wall)] -- 
Progress of 'loading chromatogram list':
-- done [took 0.00 s (CPU), 0.00 s (Wall)] -- 
Error: No sum formula given for target 'Unknown'; cannot calculate isotope distribution - using estimation method for peptides.
<RT window size calculated as 240 seconds.> occurred 4 times
Extracting chromatograms...
Detecting chromatographic peaks...
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 561.357276466771054.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 562.360631304571029.
Found no signal. The Gaussian width is probably smaller tha